In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score


In [9]:
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
print("First five rows of the bikes dataset:")
print(bikes.head())

print("\nDataset info:")
print(bikes.info())
print(bikes.columns)


First five rows of the bikes dataset:
     dteday   hr  casual  registered  temp_c  feels_like_c     hum  windspeed  \
0  1/1/2011  0.0       3          13     3.0           3.0  0.7957        0.8   
1  1/1/2011  1.0       8          30     1.7           1.7  0.8272        0.8   
2  1/1/2011  2.0       5          26     1.9           1.9  0.8157        1.1   
3  1/1/2011  3.0       3           9     2.5           2.5  0.7831        0.8   
4  1/1/2011  4.0       0           1     2.0           2.0  0.8075        1.1   

   weathersit  season  holiday  workingday  
0           1       1        0           0  
1           1       1        0           0  
2           1       1        0           0  
3           1       1        0           0  
4           1       1        0           0  

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112475 entries, 0 to 112474
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------  

In [10]:
target_column = 'registered'

# Drop 'dteday' since it's an object and not directly useful without further processing.
# You could also extract useful features from it if needed.
X = bikes.drop(columns=[target_column, 'dteday'])
y = bikes[target_column]

print("\nProcessed feature columns:")
print(X.head())


Processed feature columns:
    hr  casual  temp_c  feels_like_c     hum  windspeed  weathersit  season  \
0  0.0       3     3.0           3.0  0.7957        0.8           1       1   
1  1.0       8     1.7           1.7  0.8272        0.8           1       1   
2  2.0       5     1.9           1.9  0.8157        1.1           1       1   
3  3.0       3     2.5           2.5  0.7831        0.8           1       1   
4  4.0       0     2.0           2.0  0.8075        1.1           1       1   

   holiday  workingday  
0        0           0  
1        0           0  
2        0           0  
3        0           0  
4        0           0  


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_train

,registered
95962,100
34637,173
5954,0
109932,19
103772,782
...,...
76820,98
110268,29
103694,6
860,62


In [12]:
# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train = norm.transform(X_train)

# transform testing dataabs
X_test = norm.transform(X_test)

model = Sequential()
model.add(Dense(128, input_dim=len(X_train[0]), activation='relu'))
model.add(Dropout(.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='leaky_relu'))

model.add(Dense(1, activation='relu'))

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 128)                 │           1,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,945 (199.00 KB)

 Trainable params: 50,945 (199.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
opt = keras.optimizers.Adam()
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])
early_stop = keras.callbacks.EarlyStopping(monitor='val_mse', patience=30)

history = model.fit(X_train, y_train, epochs=2000, validation_split=.35, batch_size=20, callbacks=[early_stop],shuffle=False)


predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print("\nTest RMSE:", rmse)
print("Test R2:", r2)

Epoch 1/2000
 541/2559 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 38106.9102 - mse: 38106.9102

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=y_test, y=predictions.flatten())
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted Bike Rentals')
plt.show()

# Plot training and validation loss (MSE) over epochs
plt.figure(figsize=(8,6))
plt.plot(history.history['mse'], label='Train MSE')
plt.plot(history.history['val_mse'], label='Validation MSE')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.title('Training History')
plt.legend()
plt.show()